In [38]:
import findspark
findspark.init()

from pyspark.sql import SparkSession
from pyspark import SparkConf, SparkContext
from pyspark.streaming import StreamingContext
from pyspark.sql.functions import *
from pyspark.sql.functions import split,col
from pyspark.sql.types import Row
from pyspark.sql.types import *
import json
from pyspark.sql.context import SQLContext
from sklearn.feature_extraction.text import HashingVectorizer
from sklearn.linear_model import PassiveAggressiveClassifier
from sklearn.linear_model import Perceptron, SGDClassifier
from sklearn.naive_bayes import MultinomialNB,BernoulliNB

In [2]:
spark = SparkSession.builder.appName("Tweet analysis").getOrCreate()

text_schema = StructType([StructField('text', StringType(), True)])
json_schema = MapType(StringType(), StringType())

tweet_df= spark \
    .readStream \
    .format("socket")\
    .option("host", "localhost") \
    .option("port",6100) \
    .load()
tweet_df.printSchema()
tweet_df1= tweet_df.selectExpr("CAST(value AS STRING)")

tweetdf2=tweet_df1.withColumn('json',from_json(col('value'),json_schema)).select(explode(col('json')))
tweetdf3=tweetdf2.withColumn('value',from_json(col('value'),json_schema)).withColumn('feature0',col('value.feature0')).withColumn('feature1',col('value.feature1')).drop('value')
tweetdf3.printSchema()

21/12/05 18:06:26 WARN Utils: Your hostname, pes1ug19cs384-VirtualBox resolves to a loopback address: 127.0.1.1; using 10.0.2.15 instead (on interface enp0s3)
21/12/05 18:06:26 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
21/12/05 18:06:26 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
21/12/05 18:06:30 WARN TextSocketSourceProvider: The socket source should not be used for production applications! It does not support recovery.


root
 |-- value: string (nullable = true)

root
 |-- key: string (nullable = false)
 |-- feature0: string (nullable = true)
 |-- feature1: string (nullable = true)



In [3]:
def pre_process(tdf):
    tdf = tdf.na.replace('', None)
    tdf = tdf.na.drop()       
    tdf = tdf.withColumn('feature1', regexp_replace('feature1',r'@\w+',''))
    tdf = tdf.withColumn('feature1', regexp_replace('feature1','[^A-Za-z0-9\s]',''))
    tdf = tdf.withColumn('feature1', regexp_replace('feature1',r'http\S+',''))
    return tdf

df = pre_process(tweetdf3)

In [39]:
''' tokenizer = Tokenizer(inputCol="feature1", outputCol="words") 
hashtf = HashingTF(numFeatures=2**16, inputCol="words", outputCol='tf') 
idf = IDF(inputCol='tf', outputCol="features", minDocFreq=50)
label_stringIdx = StringIndexer(inputCol = "feature0", outputCol = "label") 
pipeline = Pipeline(stages=[tokenizer, hashtf, idf, label_stringIdx])

pipelineFit = pipeline.fit(df) ddf = pipelineFit.transform(df)
ddf.printSchema()

LR = LogisticRegression(featuresCol = 'feature1', labelCol = 'feature0', maxIter=15) 
LR_model = LR.fit(df) trainingSummary = LR_model.summary 
tk = Tokenizer(inputCol="feature1", outputCol="words")
    b = tk.transform(b) 
    hf = HashingTF(numFeatures=2, inputCol="words", outputCol='tf') 
    b = hf.transform(b)
import sklearn.feature_extraction.text.HashingVectorizer''' 

partial_fit_classifiers = {
    'Perceptron': Perceptron(),
    'Passive-Aggressive': PassiveAggressiveClassifier(),
}

num=0
model1 = BernoulliNB()
model2 = Perceptron(alpha=0.4)
model3 = SGDClassifier()
def mab(b):
    global num
    num+=1
    arr = [row['feature1'] for row in b.collect()]
    tar = [row['feature0'] for row in b.collect()]
    vectorizer = HashingVectorizer( n_features=2**15)
    x=None
    if arr:
        x = vectorizer.transform(arr)
        if num<=10:
            model1.partial_fit(x, tar,classes=['0','4'])
            model2.partial_fit(x,tar,classes=['0','4'])
            model3.partial_fit(x,tar,classes=['0','4'])
        else:
            acc1 = model1.score(x,tar)
            acc2 = model2.score(x,tar)
            acc3 = model3.score(x,tar)
            print((acc1*100, acc2*100, acc3*100))

def p(b,id):    
    mab(b)

query = df.writeStream.foreachBatch(p).start()
query.awaitTermination

21/12/05 19:15:16 WARN StreamingQueryManager: Temporary checkpoint location created which is deleted normally when the query didn't fail: /tmp/temporary-e7cde65b-fbc4-49db-93bc-21f37e06127c. If it's required to delete it under any circumstances, please set spark.sql.streaming.forceDeleteTempCheckpointLocation to true. Important to know deleting temp checkpoint folder is best effort.


<bound method StreamingQuery.awaitTermination of <pyspark.sql.streaming.StreamingQuery object at 0x7fadd5544670>>

NameError: name 'tran_df' is not defined

In [12]:


query = tran_df.writeStream.format("console").option('truncate',False).start()
query.awaitTermination()

NameError: name 'tran_df' is not defined